## NetLogo interaction through the pyNetLogo connector

This notebook provides a simple example of interaction between a NetLogo model and the Python environment, using the Wolf Sheep Predation model included in the NetLogo example library (Wilensky, 1999). A more detailed description of the different commands is provided in the paper.

We start by instantiating a link to NetLogo, loading the model, and running basic commands to set up the model and run it for 100 ticks.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import jpype
import pyNetLogo

netlogo = pyNetLogo.NetLogoLink(gui=True)

#By default, the NetLogoLink object will attempt to find a NetLogo installation. We can also specify a custom directory
#and version (currently either 5.2 or 6.0):

#netlogo = pyNetLogo.NetLogoLink(gui=True, nl_dir=r'C:\Program Files\NetLogo 6.0-BETA1', nl_version='6.0')
#netlogo = pyNetLogo.NetLogoLink(gui=True, nl_dir=r'C:\Program Files (x86)\NetLogo 5.2.1', nl_version='5.2')

netlogo.load_model(r'/Wolf Sheep Predation.nlogo')
netlogo.command('setup')

#We can use either of the following commands to run the model for 100 ticks:
netlogo.command('repeat 100 [go]')
#netlogo.repeat_command('go', 100)

The report function is then used to return arrays to the Python workspace, containing the xcor and ycor coordinates of the "sheep" agents sorted by their "who" number. An additional array is also returned to list the energy of the corresponding agents.

The report function directly passes a string to the NetLogo instance, so that the command syntax may need to be adjusted depending on the version. The NL_VERSION property of the link object can be used to check the current version.

In [ ]:
if netlogo.NL_VERSION == '6.0':
    x = netlogo.report('map [[?1] -> [xcor] of ?1] sort sheep')
    y = netlogo.report('map [[?1] -> [ycor] of ?1] sort sheep')
    energy = netlogo.report('map [[?1] -> [energy] of ?1] sort sheep')
else:
    x = netlogo.report('map [[xcor] of ?1] sort sheep')
    y = netlogo.report('map [[ycor] of ?1] sort sheep')
    energy = netlogo.report('map [[energy] of ?1] sort sheep')

We can then plot the coordinates of the agents on a scatter plot, with an additional color dimension to illustrate each agent's energy value.

Similarly, still using the basic report function, we can generate histograms from the arrays containing energy values for the sheep and wolf agents.

In [ ]:
sns.set_style('white')

fig, ax = plt.subplots(1, 2)
sc = ax[0].scatter(x, y, s=50, c=energy, cmap=plt.cm.coolwarm)
ax[0].set_xlabel('xcor')
ax[0].set_ylabel('ycor')
cbar = fig.colorbar(sc, ax=ax[0])
cbar.set_label('Energy of sheep')

energy_1 = netlogo.report('[energy] of sheep')
energy_2 = netlogo.report('[energy] of wolves')

sns.distplot(energy_1, kde=False, ax=ax[1], label='Sheep')
sns.distplot(energy_2, kde=False, ax=ax[1], label='Wolves')
ax[1].set_xlabel('Energy')
plt.legend()
plt.show()

The repeat_report function returns a Pandas dataframe containing reported values over a given number of ticks, for one or multiple reporters. The dataframe is indexed by ticks, with labeled columns for each reporter. In this case, we track the number of wolf and sheep agents over 200 ticks; the outcomes are first plotted as a function of time. The number of wolf agents is then plotted as a function of the number of sheep agents, to approximate a phase-space plot. 

In [ ]:
counts = netlogo.repeat_report(['count wolves','count sheep'], 200)

fig, ax = plt.subplots(1, 2)
counts[['count wolves','count sheep']].plot(x=counts.index, ax=ax[0])
ax[0].set_xlabel('Ticks')
ax[1].plot(counts['count wolves'], counts['count sheep'])
ax[1].set_xlabel('Wolves')
ax[1].set_ylabel('Sheep')
plt.show()

The repeat_report function can also be used with reporters that return an array. In this case, we track the energy of the wolf and sheep agents over 5 ticks, and plot the distribution of the wolves' energy at the final tick recorded in the dataframe.

In [ ]:
energy_df = netlogo.repeat_report(['[energy] of wolves', '[energy] of sheep'], 5)

fig, ax = plt.subplots(1)
sns.distplot(energy_df['[energy] of wolves'].iloc[-1], kde=False, ax=ax)
ax.set_xlabel('Energy')
plt.show()

Finally, the patch_report function is used to return a dataframe which (for this example) contains the 'pcolor' attribute of each NetLogo patch. This dataframe essentially replicates the NetLogo environment, with column labels corresponding to the xcor patch coordinates, and indices following the pycor coordinates.

In [ ]:
pcolor_df = netlogo.patch_report('pcolor')

fig, ax = plt.subplots(1)
patches = sns.heatmap(pcolor_df, xticklabels=5, yticklabels=5, cbar_kws={'label':'pcolor'}, ax=ax)
ax.set_xlabel('pxcor')
ax.set_ylabel('pycor')
plt.show()

The dataframes can be manipulated with any of the existing Pandas functions, for instance by exporting to an Excel file. The patch_set function provides the inverse functionality to patch_report, and updates the NetLogo environment from a dataframe.

In [ ]:
pcolor_df.to_excel('pcolor.xlsx')
netlogo.patch_set('pcolor', pcolor_df/2)

Finally, the kill_workspace() function shuts down the NetLogo instance.

In [ ]:
netlogo.kill_workspace()